In [15]:
import pandas as pd
import numpy as np
import joblib
import logging
import time

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin

import operators


class SimpleCategoricalImputer(BaseEstimator, TransformerMixin):

    def __init__(self, variables, fill_value="Missing"):
        self.variables = variables
        self.fill_value = fill_value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for var in self.variables:
            if var in X.columns:
                X[var] = X[var].fillna(self.fill_value).astype(str)
        return X


In [17]:
FINAL_PIPELINE_PATH = "../models/sales_pipeline.pkl"
sales_pipeline = joblib.load(FINAL_PIPELINE_PATH)
print("Pipeline cargado.")


Pipeline cargado.


In [ ]:
# 3. Cargar dataset original y definir el set de testing (20% final)

dataset = pd.read_csv("../data/raw/train.csv")


dataset["date"] = pd.to_datetime(dataset["date"])
dataset = dataset.sort_values("date").reset_index(drop=True)


dataset["year"] = dataset["date"].dt.year
dataset["month"] = dataset["date"].dt.month
dataset["day_of_week_name"] = dataset["date"].dt.day_name()

# tipos categóricos igual que en entrenamiento
dataset["store"] = dataset["store"].astype("O")
dataset["item"] = dataset["item"].astype("O")
dataset["day_of_week_name"] = dataset["day_of_week_name"].astype("O")


FEATURES = ["store", "item", "year", "month", "day_of_week_name"]
TARGET = "sales"

n_samples = len(dataset)
n_train = int(n_samples * 0.8)

print("Total registros:", n_samples)
print("Índice corte train/test:", n_train)


test_df = dataset.iloc[n_train:].copy()

print("Shape test_df:", test_df.shape)
test_df.head()


Total registros: 913000
Índice corte train/test: 730400
Shape test_df: (182600, 7)


,date,store,item,sales,year,month,day_of_week_name
730400,2016-12-31,5,19,23,2016,12,Saturday
730401,2016-12-31,6,14,31,2016,12,Saturday
730402,2016-12-31,8,20,63,2016,12,Saturday
730403,2016-12-31,1,12,48,2016,12,Saturday
730404,2016-12-31,10,37,34,2016,12,Saturday


In [23]:
FEATURES = ["store", "item", "year", "month", "day_of_week_name"]
X_test = test_df[FEATURES].copy()


In [24]:
y_test_pred = sales_pipeline.predict(X_test)

print("Predicciones generadas:", len(y_test_pred))
y_test_pred[:10]


C:\Users\leidb\AppData\Local\Temp\ipykernel_39180\232181804.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
C:\Users\leidb\AppData\Local\Temp\ipykernel_39180\232181804.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


Predicciones generadas: 182600


array([48.02970071, 48.02970071, 48.02970071, 48.02970071, 48.02970071,
       48.02970071, 48.02970071, 48.02970071, 48.02970071, 48.02970071])

In [25]:
output_df = test_df.copy()
output_df["sales_pred"] = y_test_pred

OUTPUT_PATH = "../data/raw/test_predictions.csv"
import os
os.makedirs("../data/processed", exist_ok=True)
output_df.to_csv(OUTPUT_PATH, index=False)

print("Archivo guardado en:", OUTPUT_PATH)
output_df.head()


Archivo guardado en: ../data/raw/test_predictions.csv


,date,store,item,sales,year,month,day_of_week_name,sales_pred
730400,2016-12-31,5,19,23,2016,12,Saturday,48.029701
730401,2016-12-31,6,14,31,2016,12,Saturday,48.029701
730402,2016-12-31,8,20,63,2016,12,Saturday,48.029701
730403,2016-12-31,1,12,48,2016,12,Saturday,48.029701
730404,2016-12-31,10,37,34,2016,12,Saturday,48.029701
